In [1]:
import numpy as np
import torch
import torch.nn as nn
from scipy import interpolate
import scipy
import matplotlib.pyplot as plt
from matplotlib import ticker, cm
from tqdm import tqdm

from vanilla_fem import *
from pod_fem import *
from deim_fem import *
from neim_fem3 import *
from other_functions import *

## Parameters
The parameters which we will let vary are $a$ so that
$$
    (a,) \in \mathbb{P} = [-0.5, 0.5].
$$

In [2]:
parameters = []
for a in np.linspace(-0.5, 0.5, 10):
    parameters.append((a,))

b = 0.5
c = 1
A0 = 500
M = 1
L = 0.1

# number of time steps and final time
Nt = 80
t_final = 4
tVals = np.linspace(0, t_final, Nt)
dt = tVals[1] - tVals[0] # delta t

In [3]:
# define the mesh in space
mesh, interior_point_coords = get_triangulation(refinements=5)

# Get the finite element space, stiffness matrix, lumped mass matrix as a vector, and mass matrix
Vh, stiffness_matrix, gamma, mass_matrix = calculate_basis_integrals(mesh)

## Solve High Fidelity PDE
Note that we only keep track of nodes in the interior of $\Omega$ because we use homogeneous Dirichlet boundary conditions.

In [4]:
hf_solutions = []
for (a,) in parameters:
    # Get vectors for the components of matrices (flattened over space) 
    # over time with time 0 initialized.
    Q1, Q2, p1, p2, r, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)

    # solve the PDE (updates the entries of Q1, Q2, etc. within the function)
    solve_Q_flow(Q1, Q2, p1, p2, r, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)
    
    hf_solutions.append((Q1, Q2, p1, p2, r))

Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 138.00it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 146.24it/s]


Computing scheme...


...

100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 147.11it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 146.66it/s]


Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 148.97it/s]


## Compute POD Matrices

In [5]:
Q1 = []
Q2 = []
p1 = []
p2 = []
r  = []
for sol in hf_solutions:
    Q1.append(sol[0])
    Q2.append(sol[1])
    p1.append(sol[2])
    p2.append(sol[3])
    r.append(sol[4])

Q1 = np.concatenate(Q1, axis=0)
Q2 = np.concatenate(Q2, axis=0)
p1 = np.concatenate(p1, axis=0)
p2 = np.concatenate(p2, axis=0)
r  = np.concatenate(r, axis=0)

max_rank = 5
U_Q1, U_Q2, U_r = get_POD_matrices(Q1, Q2, r, max_rank)

## Compare POD Solution to High Fidelity Solution (New Parameter)

In [6]:
a = -0.2

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod = initialize_Q_flow_POD(Q1_, Q2_, p1_, p2_, r_, U_Q1, U_Q2, U_r)
solve_Q_flow_POD(Q1_pod, Q2_pod, p1_pod, p2_pod, r_pod, gamma, stiffness_matrix, U_Q1, U_Q2, U_r, Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_pod.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 145.03it/s]


Computing scheme...


100%|████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 1357.07it/s]


Relative Error: 0.019050948314289433


## Compute DEIM Operators

In [7]:
deim_modes = 7

nonlinearityQ = M * gamma.reshape(-1, 1) * np.concatenate([p1 * r, p2 * r], axis=0).T
UN_Q, nQ_indices = get_DEIM_operators(nonlinearityQ, deim_modes)
U_deimQ1 = U_Q1.T @ UN_Q
U_deimQ2 = U_Q2.T @ UN_Q

nonlinearityR = (2 * p1[:-1] * (Q1[1:] - Q1[:-1]) + 2 * p2[:-1] * (Q2[1:] - Q2[:-1])).T
UN_R, nR_indices = get_DEIM_operators(nonlinearityR, deim_modes)
U_deimR = U_r.T @ UN_R

## Compare DEIM Solution to High Fidelity Solution (New Parameter)

In [8]:
a = -0.15

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim = initialize_Q_flow_DEIM(Q1_, Q2_, p1_, p2_, r_, 
                                                                                          U_Q1, U_Q2, U_r, 
                                                                                          nQ_indices, nR_indices)
solve_Q_flow_DEIM(Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim, 
                    gamma, stiffness_matrix, 
                    U_Q1, U_Q2, U_r, U_deimQ1, U_deimQ2, U_deimR,
                    nQ_indices, nR_indices,
                    Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_deim.T).T) / np.linalg.norm(Q1_))

Computing scheme...


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 130.80it/s]


Computing scheme...


100%|████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 3949.16it/s]


Relative Error: 0.02106035786683805


## Compute NEIM Operators

### $Q$ Equation NEIM

TODO: Generalize so that we only use data within 10 indices in time for each example (to cut down on computation time and reduce required capacity for neural nets)

In [9]:
time_frequency = 1
mu = []
"""
for (a, b, c) in parameters:
    for t in range(0, 5, time_frequency):#Nt, time_frequency):
        mu.append([t, a, b, c])
"""
for (a,) in parameters:
    for t in range(0, Nt, time_frequency):
        mu.append([t, a])
mu = np.array(mu)

num_time_params = len(range(0, Nt, time_frequency))

In [ ]:
time_window = 10
max_time = Nt

ro_sols = np.zeros((mu.shape[0], time_window, 3*max_rank))
f_NEIM  = np.zeros((mu.shape[0], time_window, 2*max_rank))

# iterate over solution parameter
for i, param1 in enumerate(mu):
    Q1_, Q2_, _, _, r_ = hf_solutions[i // num_time_params]
    Q1_, Q2_, r_ = Q1_[i % num_time_params], Q2_[i % num_time_params], r_[i % num_time_params]
    
    ro_sol = np.concatenate([Q1_ @ U_Q1, Q2_ @ U_Q2, r_ @ U_r])
    
    time_i = int(mu[i, 0] + 0.5)
    time_j = 0
    
    # iterate over nonlinearity parameter
    for j, param2 in enumerate(mu):
        # evaluate Nonlinearity(solution(param1); param2)
        if (i - j >= 0 and i - j <= time_i) or (i - j <= 0 and j - i <= max_time - time_i):
            (t, a) = param2

            Q = np.concatenate((
                    np.concatenate((Q1_[:, None, None], Q2_[:, None, None]), axis=2),
                    np.concatenate((Q2_[:, None, None], -Q1_[:, None, None]), axis=2),
            ), axis=1)
            PQ = P(Q, a, b, c, A0)
            p1_ = PQ[:, 0, 0]
            p2_ = PQ[:, 0, 1]

            nonlinearityQ = M * np.concatenate([(gamma[None] * p1_ * r_) @ U_Q1, (gamma[None] * p2_ * r_) @ U_Q2], axis=1)
            ro_sols[i, time_j] = ro_sol
            f_NEIM[i, time_j] = nonlinearityQ.reshape(-1)
            time_j += 1

In [ ]:
NQ_NEIM, _, _, _, _ = NEIM_Q(ro_sols, f_NEIM, mu, max_modes=7, train_loop_iterations=20000, theta_train_loop_iterations=30000)

### $r$ Equation NEIM

In [ ]:
time_frequency = 1
mu = []
"""
for (a, b, c) in parameters:
    for t in range(0, 5-1, time_frequency):#Nt-1, time_frequency):
        mu.append([t, a, b, c])
"""
for (a,) in parameters:
    for t in range(0, Nt-1, time_frequency):
        mu.append([t, a])
mu = np.array(mu)

num_time_params = len(range(0, Nt-1, time_frequency))

In [ ]:
ro_sols = np.zeros((mu.shape[0], 4*max_rank))
f_NEIM  = np.zeros((mu.shape[0], mu.shape[0], max_rank))

# iterate over solution parameter
for i, param1 in enumerate(mu):
    Q1_, Q2_, _, _, r_ = hf_solutions[i // num_time_params]
    time_idx = i % num_time_params
    
    Q1_0, Q2_0, r_0 = Q1_[time_idx], Q2_[time_idx], r_[time_idx]
    Q1_1, Q2_1, r_1 = Q1_[time_idx+1], Q2_[time_idx+1], r_[time_idx+1]
    
    ro_sols[i] = np.concatenate([Q1_1 @ U_Q1, Q1_0 @ U_Q1, Q2_1 @ U_Q2, Q2_0 @ U_Q2])
    
    # iterate over nonlinearity parameter
    for j, param2 in enumerate(mu):
        # evaluate Nonlinearity(solution(param1); param2)
        (t, a) = param2
        
        Q = np.concatenate((
                np.concatenate((Q1_0[:, None, None], Q2_0[:, None, None]), axis=2),
                np.concatenate((Q2_0[:, None, None], -Q1_0[:, None, None]), axis=2),
        ), axis=1)
        PQ = P(Q, a, b, c, A0)
        p1_ = PQ[:, 0, 0]
        p2_ = PQ[:, 0, 1]
        
        nonlinearityR = (2 * p1_ * (Q1_1 - Q1_0) + 2 * p2_ * (Q2_1 - Q2_0)) @ U_r
        f_NEIM[i, j] = nonlinearityR.reshape(-1)

In [ ]:
NR_NEIM, _, _, _, _ = NEIM_R(ro_sols, f_NEIM, mu, max_modes=7, train_loop_iterations=20000, theta_train_loop_iterations=30000)

## Compare NEIM Solution to High Fidelity Solution (New Parameter)

In [ ]:
a = -0.15

Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

with torch.no_grad():

    Q1_neim, Q2_neim, r_neim = initialize_Q_flow_NEIM(Q1_, Q2_, r_, U_Q1, U_Q2, U_r)

    solve_Q_flow_NEIM(Q1_neim, Q2_neim, r_neim, 
                        gamma, stiffness_matrix, 
                        U_Q1, U_Q2, NQ_NEIM, NR_NEIM,
                        Nt, a, b, c, A0, M, L, dt)

print("Relative Error:", np.linalg.norm(Q1_ - (U_Q1 @ Q1_neim.T).T) / np.linalg.norm(Q1_))

## DEIM-NEIM Comparison

In [ ]:
def solve_deim_neim(a=0., num_modes=-1):
    assert -1 <= num_modes and num_modes <= deim_modes
    if num_modes == -1:
        num_modes = deim_modes
    

    Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
    solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)
    
    nonlinearityQ = M * gamma.reshape(-1, 1) * np.concatenate([p1 * r, p2 * r], axis=0).T
    UN_Q, nQ_indices = get_DEIM_operators(nonlinearityQ, num_modes)
    U_deimQ1 = U_Q1.T @ UN_Q
    U_deimQ2 = U_Q2.T @ UN_Q

    nonlinearityR = (2 * p1[:-1] * (Q1[1:] - Q1[:-1]) + 2 * p2[:-1] * (Q2[1:] - Q2[:-1])).T
    UN_R, nR_indices = get_DEIM_operators(nonlinearityR, num_modes)
    U_deimR = U_r.T @ UN_R

    Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim = initialize_Q_flow_DEIM(Q1_, Q2_, p1_, p2_, r_, 
                                                                                              U_Q1, U_Q2, U_r, 
                                                                                              nQ_indices[:num_modes], 
                                                                                              nR_indices[:num_modes])
    
    solve_Q_flow_DEIM(Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim, 
                        gamma, stiffness_matrix, 
                        U_Q1, U_Q2, U_r, U_deimQ1, U_deimQ2, U_deimR,
                        nQ_indices[:num_modes], nR_indices[:num_modes],
                        Nt, a, b, c, A0, M, L, dt)

    print("Relative Error (DEIM):", np.linalg.norm(Q1_ - (U_Q1 @ Q1_deim.T).T))# / np.linalg.norm(Q1_))


    with torch.no_grad():

        Q1_neim, Q2_neim, r_neim = initialize_Q_flow_NEIM(Q1_, Q2_, r_, U_Q1, U_Q2, U_r)

        solve_Q_flow_NEIM(Q1_neim, Q2_neim, r_neim, 
                            gamma, stiffness_matrix, 
                            U_Q1, U_Q2, lambda x, y: NQ_NEIM(x, y, num_modes), lambda x, y: NR_NEIM(x, y, num_modes),
                            Nt, a, b, c, A0, M, L, dt)

    print("Relative Error (NEIM):", np.linalg.norm(Q1_ - (U_Q1 @ Q1_neim.T).T))# / np.linalg.norm(Q1_))
    
    return Q1_, Q2_, r_, Q1_deim, Q2_deim, r_deim, Q1_neim, Q2_neim, r_neim

### Relative Error Over Time for Fixed $a$

In [ ]:
Q1_, Q2_, r_, Q1_deim, Q2_deim, r_deim, Q1_neim, Q2_neim, r_neim = solve_deim_neim(0.0, 7)

In [ ]:
error_time_neim = [np.linalg.norm(Q1_[time] - (U_Q1 @ Q1_neim.T).T[time]) / np.linalg.norm(Q1_[time]) for time in range(Q1_.shape[0])]

In [ ]:
error_time_deim = [np.linalg.norm(Q1_[time] - (U_Q1 @ Q1_deim.T).T[time]) / np.linalg.norm(Q1_[time]) for time in range(Q1_.shape[0])]

In [ ]:
plt.semilogy(tVals[:10], error_time_neim[:10], label="NEIM")
plt.semilogy(tVals[:10], error_time_deim[:10], label="DEIM")
plt.xlabel("$t$")
plt.ylabel("$Q_{11}$ Relative Error")
plt.legend()
plt.show()

In [ ]:
error_time_neim[:10], np.mean(error_time_neim[:10])

### Total Relative Error Over $a$

In [ ]:
a_list = np.arange(-0.5, 0.55, 0.05)
error_deim = []
error_neim = []
for a in a_list:
    Q1_, Q2_, r_, Q1_deim, Q2_deim, r_deim, Q1_neim, Q2_neim, r_neim = solve_deim_neim(a)
    error_deim.append(np.linalg.norm(Q1_ - (U_Q1 @ Q1_deim.T).T) / np.linalg.norm(Q1_))
    error_neim.append(np.linalg.norm(Q1_ - (U_Q1 @ Q1_neim.T).T) / np.linalg.norm(Q1_))

In [ ]:
plt.semilogy(a_list, error_deim, label="DEIM")
plt.semilogy(a_list, error_neim, label="NEIM")
plt.xlabel("$a$")
plt.ylabel("$Q_{11}$ Relative Error")
plt.legend()
plt.show()

### DEIM-NEIM Error Contour Plots for $Q_{11}$

In [ ]:
a_list = np.arange(-0.5, 0.55, 0.05)
error_deim_contour = []
error_neim_contour = []
for a in a_list:
    error_deim_contour.append([])
    error_neim_contour.append([])
    
    Q1_, Q2_, r_, Q1_deim, Q2_deim, r_deim, Q1_neim, Q2_neim, r_neim = solve_deim_neim(a)
    
    for i, t in enumerate(tVals):
        error_deim_contour[-1].append(np.linalg.norm(Q1_[i] - (U_Q1 @ Q1_deim.T).T[i]) / np.linalg.norm(Q1_[i]))
        error_neim_contour[-1].append(np.linalg.norm(Q1_[i] - (U_Q1 @ Q1_neim.T).T[i]) / np.linalg.norm(Q1_[i]))

a_contour, time_contour = np.meshgrid(tVals, a_list)

In [ ]:
fig, ax = plt.subplots(1, 2)

ax[0].contourf(a_contour, time_contour, error_deim_contour, locator=ticker.LogLocator(), vmin=np.min(error_deim_contour), vmax=np.max(error_neim_contour))
ax[0].set_xlabel("Time $t$")
ax[0].set_ylabel("$a$")
ax[0].set_title("DEIM Relative Error")

cs = ax[1].contourf(a_contour, time_contour, error_neim_contour, locator=ticker.LogLocator())
ax[1].set_xlabel("Time $t$")
ax[1].set_title("NEIM Relative Error")
ax[1].set_xlabel("Time $t$")
ax[1].scatter(np.array([9, 10, 0, 11, 15, 1, 8])*dt, np.array([0.5, 0.5, -0.5, 0.5, -0.5, -0.5, 0.388888888888]), s=100, color='r', marker="*")
ax[1].set_xlim(ax[0].get_xlim())
ax[1].set_ylim(ax[0].get_ylim())
plt.colorbar(cs)
plt.show()

In [ ]:
mu[720], mu[721], mu[0], mu[722], mu[15], mu[1], mu[640]

### Error in Nonlinear Term (Not Error of Solution)

In [ ]:
def solve_deim_neim_nonlinearities(a=0.):

    Q1_, Q2_, p1_, p2_, r_, num_interior_points = initialize_Q_flow(interior_point_coords, Nt, a, b, c, A0)
    solve_Q_flow(Q1_, Q2_, p1_, p2_, r_, gamma, stiffness_matrix, Nt, a, b, c, A0, M, L, dt)

    Q1_deim, Q2_deim, p1_deimQ, p2_deimQ, p1_deimR, p2_deimR, r_deim = initialize_Q_flow_DEIM(Q1_, Q2_, p1_, p2_, r_, 
                                                                                              U_Q1, U_Q2, U_r, 
                                                                                              nQ_indices, nR_indices)
    hfQ_nonlinearities = []
    hfR_nonlinearities = []
    deimQ_nonlinearities = []
    deimR_nonlinearities = []
    neimQ_nonlinearities = []
    neimR_nonlinearities = []
    for t in range(Q1_.shape[0]-1):
        
        g1 = M * gamma * p1_[t] * r_[t]
        g2 = M * gamma * p2_[t] * r_[t]
        Q_nonlinearity = np.concatenate((g1, g2))
        R_nonlinearity = 2 * p1_[t] * (Q1_[t+1] - Q1_[t]) + 2 * p2_[t] * (Q2_[t+1] - Q2_[t])
        
        hfQ_nonlinearities.append(Q_nonlinearity)
        hfR_nonlinearities.append(R_nonlinearity)
        
        Q1_t = (U_Q1 @ U_Q1.T @ Q1_[t].reshape(-1, 1)).reshape(-1)
        Q2_t = (U_Q2 @ U_Q2.T @ Q2_[t].reshape(-1, 1)).reshape(-1)
        
        Q1_t1 = (U_Q1 @ U_Q1.T @ Q1_[t+1].reshape(-1, 1)).reshape(-1)
        Q2_t1 = (U_Q2 @ U_Q2.T @ Q2_[t+1].reshape(-1, 1)).reshape(-1)
        
        Q_t = np.concatenate((
                    np.concatenate((Q1_t[:, None, None], Q2_t[:, None, None]), axis=2),
                    np.concatenate((Q2_t[:, None, None], -Q1_t[:, None, None]), axis=2),
            ), axis=1)
        PQ = P(Q_t, a, b, c, A0)
        p1_t = PQ[:, 0, 0]
        p2_t = PQ[:, 0, 1]
        
        g1 = M * U_Q1 @ U_deimQ1 @ (gamma[nQ_indices].reshape(-1, 1) * p1_t[nQ_indices].reshape(-1, 1) * (U_r[nQ_indices] @ U_r.T @ r_[t].reshape(-1, 1)))
        g2 = M * U_Q2 @ U_deimQ2 @ (gamma[nQ_indices].reshape(-1, 1) * p2_t[nQ_indices].reshape(-1, 1) * (U_r[nQ_indices] @ U_r.T @ r_[t].reshape(-1, 1)))
    
        deimQ_nonlinearity = np.concatenate((g1.reshape(-1), g2.reshape(-1)))
        deimR_nonlinearity = (U_r @ (U_deimR @ (2*(p1_t * (Q1_t1 - Q1_t))[nR_indices].reshape(-1, 1)\
                             + 2*(p2_t * (Q2_t1 - Q2_t))[nR_indices].reshape(-1, 1)))).reshape(-1)
        
        deimQ_nonlinearities.append(deimQ_nonlinearity)
        deimR_nonlinearities.append(deimR_nonlinearity)

        with torch.no_grad():
            ro_sol = np.concatenate([U_Q1.T @ Q1_[t].reshape(-1, 1), U_Q2.T @ Q2_[t].reshape(-1, 1), U_r.T @ r_[t].reshape(-1, 1)], axis=0).reshape(-1)
            neimQ_nonlinearity = (scipy.linalg.block_diag(U_Q1, U_Q2) @ NQ_NEIM(np.array([t, a]), ro_sol).reshape(-1, 1)).reshape(-1)
            ro_sol = np.concatenate([U_Q1.T @ Q1_[t].reshape(-1, 1), U_Q1.T @ Q1_[t-1].reshape(-1, 1), 
                                     U_Q2.T @ Q2_[t].reshape(-1, 1), U_Q2.T @ Q2_[t-1].reshape(-1, 1)], axis=0).reshape(-1)
            neimR_nonlinearity = (U_r @ NR_NEIM(np.array([t-1, a]), ro_sol).reshape(-1, 1)).reshape(-1)

            neimQ_nonlinearities.append(neimQ_nonlinearity)
            neimR_nonlinearities.append(neimR_nonlinearity)
    
    hfQ_nonlinearities = np.array(hfQ_nonlinearities)
    hfR_nonlinearities = np.array(hfR_nonlinearities)
    deimQ_nonlinearities = np.array(deimQ_nonlinearities)
    deimR_nonlinearities = np.array(deimR_nonlinearities)
    neimQ_nonlinearities = np.array(neimQ_nonlinearities)
    neimR_nonlinearities = np.array(neimR_nonlinearities)
    
    return hfQ_nonlinearities, hfR_nonlinearities, deimQ_nonlinearities, deimR_nonlinearities, neimQ_nonlinearities, neimR_nonlinearities


# a_list = np.arange(-0.5, 0.55, 0.05)
# error_deim_contour = []
# error_neim_contour = []
# for a in a_list:
#     error_deim_contour.append([])
#     error_neim_contour.append([])
    
#     Q1_, Q2_, r_, Q1_deim, Q2_deim, r_deim, Q1_neim, Q2_neim, r_neim = solve_deim_neim(a)
    
#     for i, t in enumerate(tVals):
#         error_deim_contour[-1].append(np.linalg.norm(Q1_[i] - (U_Q1 @ Q1_deim.T).T[i]) / np.linalg.norm(Q1_[i]))
#         error_neim_contour[-1].append(np.linalg.norm(Q1_[i] - (U_Q1 @ Q1_neim.T).T[i]) / np.linalg.norm(Q1_[i]))

# a_contour, time_contour = np.meshgrid(tVals, a_list)

In [ ]:
hfQ_nonlinearities, hfR_nonlinearities, deimQ_nonlinearities, deimR_nonlinearities, neimQ_nonlinearities, neimR_nonlinearities = solve_deim_neim_nonlinearities(a=0.15)

In [ ]:
for t in range(hfQ_nonlinearities.shape[0]):
    print("DEIM:", np.linalg.norm(hfR_nonlinearities[t] - deimR_nonlinearities[t]) / np.linalg.norm(hfR_nonlinearities[t]), "\tNEIM:", np.linalg.norm(hfR_nonlinearities[t] - neimR_nonlinearities[t]) / np.linalg.norm(hfR_nonlinearities[t]))
    print("DEIM:", np.linalg.norm(hfQ_nonlinearities[t] - deimQ_nonlinearities[t]) / np.linalg.norm(hfQ_nonlinearities[t]), "\tNEIM:", np.linalg.norm(hfQ_nonlinearities[t] - neimQ_nonlinearities[t]) / np.linalg.norm(hfQ_nonlinearities[t]))
    print("")

### Error with respect to number of modes taken

In [ ]:
f_NEIM